In [ ]:
!pip install torch torchvision transformers

     |████████████████████████████████| 4.0 MB 32.7 MB/s 
     |████████████████████████████████| 84 kB 3.6 MB/s 
     |████████████████████████████████| 596 kB 7.2 MB/s 
     |████████████████████████████████| 880 kB 53.6 MB/s 
     |████████████████████████████████| 6.6 MB 58.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=2d2838949030efd3def8cd526d083481ec1897ee46b320780e0cb26e44d5079e
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import numpy as np
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import torch
from transformers import BertTokenizer
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from transformers import BertModel

In [ ]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv('train_label.csv')
dev = pd.read_csv('dev_label.csv')
test = pd.read_csv('test.csv')

In [ ]:
train

,row,influence,role,credibility,list,profile_description,profile_url,age,text,label
0,0,8.160000e+02,0.346203,0.000000,2.000000,2.000000,0.000000,6470.000000,5. Can regularly rinsing your nose with saline...,0
1,1,5.520169e+06,6251.467708,1.709677,35395.193550,1.741935,1.709677,3999.161290,French police chief killed himself after #Char...,1
2,2,1.222000e+03,1.287671,0.000000,2.000000,2.000000,2.000000,6500.500000,Coronavirus disease (COVID-19) advice for the ...,0
3,3,3.553883e+07,33338.021080,1.800000,225507.200000,2.000000,2.000000,4970.600000,Ottawa police confirm that there were multiple...,0
4,4,4.918000e+03,0.959984,0.000000,7.000000,1.000000,1.000000,511.000000,if the primary focus of a government isn't to ...,0
...,...,...,...,...,...,...,...,...,...,...
1565,1889,1.262000e+03,1.974961,0.000000,2.000000,2.000000,0.000000,5470.000000,4. It cannot be transmitted through goods manu...,0
1566,1890,2.256051e+05,3.862789,1.571429,2000.285714,1.619048,1.571429,2269.190476,Desperate Ted Cruz Claims Planned Parenthood S...,1
1567,1891,2.778364e+07,56933.684750,1.593750,104322.093800,1.593750,1.593750,3879.312500,"""Thoughts and prayers are not enough."" Pres. O...",1
1568,1892,1.612269e+07,8672.777479,1.777778,89795.555560,1.777778,1.777778,4423.111111,Police have surrounded this building where the...,0


In [ ]:
train = train[['text', 'label']]
dev = dev[['text', 'label']]
test = test[['text']]

In [ ]:
train

,text,label
0,5. Can regularly rinsing your nose with saline...,0
1,French police chief killed himself after #Char...,1
2,Coronavirus disease (COVID-19) advice for the ...,0
3,Ottawa police confirm that there were multiple...,0
4,if the primary focus of a government isn't to ...,0
...,...,...
1565,4. It cannot be transmitted through goods manu...,0
1566,Desperate Ted Cruz Claims Planned Parenthood S...,1
1567,"""Thoughts and prayers are not enough."" Pres. O...",1
1568,Police have surrounded this building where the...,0


In [ ]:
train['text'][0]

'5. Can regularly rinsing your nose with saline help prevent infection with the new coronavirus? https://t.co/ccMjhhD7BK  4. Can eating garlic help prevent infection with the new coronavirus? #COVID19Malaysia https://t.co/q133xXBiwl 6. Do vaccines against pneumonia protect you against the new coronavirus? https://t.co/wL0mlEqU95 7. Can spraying alcohol or chlorine all over your body kill the new coronavirus? #Chamber https://t.co/zunVR7Ht0V 8. How effective are thermal scanners in detecting people infected with the new coronavirus? https://t.co/nyLOyKAb1H 9. Can an ultraviolet disinfection lamp kill the new coronavirus? https://t.co/ZrlllbkIjm 10. Are hand dryers effective in killing the new coronavirus? https://t.co/cSDKXO1bGr 11. The new coronavirus CANNOT be transmitted through mosquito bites. https://t.co/ZRL8bjRkpl 12. Taking a hot bath does not prevent the new coronavirus disease https://t.co/bICOqSTOuD 13. Cold weather and snow CANNOT kill the new coronavirus. https://t.co/7yeQQ

In [ ]:
def process_tweet(tweet):
    
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'http\S+', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            tweets_clean.append(word)
            
    tweet_clean = ' '.join(tweets_clean)

    return tweet_clean

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
processed_train = [process_tweet(t) for t in train.text]
processed_dev = [process_tweet(t) for t in dev.text]
processed_test = [process_tweet(t) for t in test.text]

In [ ]:
train.text = processed_train
dev.text = processed_dev
test.text = processed_test

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
train.text

0       5 regularly rinsing nose saline help prevent i...
1       french police chief killed charliehebdo attack...
2       coronavirus disease covid 19 advice public ✳ ️...
3       ottawa police confirm multiple suspects shooti...
4       primary focus government alleviate suffering d...
                              ...                        
1565    4 cannot transmitted goods manufactured china ...
1566    desperate ted cruz claims planned parenthood s...
1567    thoughts prayers enough pres obama speaks mass...
1568    police surrounded building suspected charliehe...
1569    help 👩 ‍ 🏫 9am socialism need bad 10am slackli...
Name: text, Length: 1570, dtype: object

In [ ]:
train.to_csv('train_text_bertplay.csv', index = False)
dev.to_csv('dev_text_bertplay.csv', index = False)
test.to_csv('test_text.csv', index = False)

In [ ]:
class SSTDataset(Dataset):

    def __init__(self, filename, maxlen):

        #Store the contents of the file in a pandas dataframe
        self.df = pd.read_csv(filename)

        #Initialize the BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        self.maxlen = maxlen

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        #Selecting the sentence and label at the specified index in the data frame
        sentence = self.df.loc[index, 'text']
        if 'label' in self.df.columns:
            label = self.df.loc[index, 'label']
        else: 
            label = None
        
        #Preprocessing the text to be suitable for BERT
        tokens = self.tokenizer.tokenize(sentence) #Tokenize the sentence
        tokens = ['[CLS]'] + tokens + ['[SEP]'] #Insering the CLS and SEP token in the beginning and end of the sentence
        if len(tokens) < self.maxlen:
            tokens = tokens + ['[PAD]' for _ in range(self.maxlen - len(tokens))] #Padding sentences
        else:
            tokens = tokens[:self.maxlen-1] + ['[SEP]'] #Prunning the list to be of specified max length

        tokens_ids = self.tokenizer.convert_tokens_to_ids(tokens) #Obtaining the indices of the tokens in the BERT Vocabulary
        tokens_ids_tensor = torch.tensor(tokens_ids) #Converting the list to a pytorch tensor

        #Obtaining the attention mask i.e a tensor containing 1s for no padded tokens and 0s for padded ones
        attn_mask = (tokens_ids_tensor != 0).long()

        if 'label' in self.df.columns:
            return tokens_ids_tensor, attn_mask, label
        else:
            return tokens_ids_tensor, attn_mask

In [ ]:
#Creating instances of training and development set
#maxlen sets the maximum length a sentence can have
#any sentence longer than this length is truncated to the maxlen size
train_set = SSTDataset(filename = 'train_text_bertplay.csv', maxlen = 200)
dev_set = SSTDataset(filename = 'dev_text_bertplay.csv', maxlen = 200)
test_set = SSTDataset(filename = 'test_text.csv', maxlen = 200)

#Creating intsances of training and development dataloaders
train_loader = DataLoader(train_set, batch_size = 64, num_workers = 2)
dev_loader = DataLoader(dev_set, batch_size = 64, num_workers = 2)
test_loader = DataLoader(test_set, batch_size = 64, num_workers = 2)

print("Done preprocessing training and development data.")

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Done preprocessing training and development data.


In [ ]:
class TextClassifier(nn.Module):

    def __init__(self):
        super(TextClassifier, self).__init__()
        #Instantiating BERT model object 
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')
        
        #Classification layer
        #input dimension is 768 because [CLS] embedding has a dimension of 768
        #output dimension is 1 because we're working with a binary classification problem
        self.cls_layer = nn.Linear(768, 1)
                         
                       

    def forward(self, seq, attn_masks):
        '''
        Inputs:
            -seq : Tensor of shape [B, T] containing token ids of sequences
            -attn_masks : Tensor of shape [B, T] containing attention masks to be used to avoid contibution of PAD tokens
        '''

        #Feeding the input to BERT model to obtain contextualized representations
        outputs = self.bert_layer(seq, attention_mask = attn_masks, return_dict=True)
        cont_reps = outputs.last_hidden_state

        #Obtaining the representation of [CLS] head (the first token)
        cls_rep = cont_reps[:, 0]

        #Feeding cls_rep to the classifier layer
        logits = self.cls_layer(cls_rep)

        return logits

In [ ]:
gpu = 0 #gpu ID

print("Creating the text classifier, initialised with pretrained BERT-BASE parameters...")
net = TextClassifier()
net.cuda(gpu)
print("Done creating the sentiment classifier.")

Creating the text classifier, initialised with pretrained BERT-BASE parameters...


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Done creating the sentiment classifier.


In [ ]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
opti = optim.Adam(net.parameters(), lr = 0.00001)

In [ ]:
import time

def train(net, criterion, opti, train_loader, dev_loader, max_eps, gpu):

    best_acc = 0
    st = time.time()
    for ep in range(max_eps):
        
        net.train()
        for it, (seq, attn_masks, labels) in enumerate(train_loader):
            #Clear gradients
            opti.zero_grad()  
            #Converting these to cuda tensors
            seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)

            #Obtaining the logits from the model
            logits = net(seq, attn_masks)

            #Computing loss
            loss = criterion(logits.squeeze(-1), labels.float())

            #Backpropagating the gradients
            loss.backward()

            #Optimization step
            opti.step()
              
            if it % 100 == 0:
                
                acc = get_accuracy_from_logits(logits, labels)
                print("Iteration {} of epoch {} complete. Loss: {}; Accuracy: {}; Time taken (s): {}".format(it, ep, loss.item(), acc, (time.time()-st)))
                st = time.time()

        
        dev_acc, dev_loss = evaluate(net, criterion, dev_loader, gpu)
        print("Epoch {} complete! Development Accuracy: {}; Development Loss: {}".format(ep, dev_acc, dev_loss))
        if dev_acc > best_acc:
            print("Best development accuracy improved from {} to {}, saving model...".format(best_acc, dev_acc))
            best_acc = dev_acc
            torch.save(net.state_dict(), 'model.dat')

In [ ]:
def get_accuracy_from_logits(logits, labels):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    soft_probs = (probs > 0.5).long()
    acc = (soft_probs.squeeze() == labels).float().mean()
    return acc

def evaluate(net, criterion, dataloader, gpu):
    net.eval()

    mean_acc, mean_loss = 0, 0
    count = 0

    with torch.no_grad():
        for seq, attn_masks, labels in dataloader:
            seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)
            logits = net(seq, attn_masks)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            mean_acc += get_accuracy_from_logits(logits, labels)
            count += 1

    return mean_acc / count, mean_loss / count

In [ ]:
num_epoch = 50

#fine-tune the model
train(net, criterion, opti, train_loader, dev_loader, num_epoch, gpu)

Iteration 0 of epoch 0 complete. Loss: 0.7977809906005859; Accuracy: 0.1875; Time taken (s): 2.64441180229187
Epoch 0 complete! Development Accuracy: 0.7786805629730225; Development Loss: 0.45321107572979397
Best development accuracy improved from 0 to 0.7786805629730225, saving model...
Iteration 0 of epoch 1 complete. Loss: 0.43289533257484436; Accuracy: 0.796875; Time taken (s): 53.246832847595215
Epoch 1 complete! Development Accuracy: 0.79625004529953; Development Loss: 0.3909488750828637
Best development accuracy improved from 0.7786805629730225 to 0.79625004529953, saving model...
Iteration 0 of epoch 2 complete. Loss: 0.32501739263534546; Accuracy: 0.828125; Time taken (s): 56.67959976196289
Epoch 2 complete! Development Accuracy: 0.8817360997200012; Development Loss: 0.3144717216491699
Best development accuracy improved from 0.79625004529953 to 0.8817360997200012, saving model...
Iteration 0 of epoch 3 complete. Loss: 0.25088492035865784; Accuracy: 0.875; Time taken (s): 61.76

In [ ]:
def predict(net, dataloader, gpu):
    net.eval()

    final_preds = []
    with torch.no_grad():
        for seq, attn_masks in dataloader:
            seq, attn_masks = seq.cuda(gpu), attn_masks.cuda(gpu)
            logits = net(seq, attn_masks)
            probs = torch.sigmoid(logits.unsqueeze(-1))
            soft_probs = (probs > 0.5).long().squeeze().cpu().detach().numpy()
            final_preds.append(soft_probs)
    final_preds = np.hstack(final_preds)

    return final_preds

In [ ]:
preds = predict(net, test_loader, gpu)

In [ ]:
results = pd.DataFrame({'Predicted': preds})
results.index.name = 'Id'
results.to_csv('v2.csv')


In [ ]:
def get_results(net, test_loader, gpu, model_path, save_path):

    net.load_state_dict(torch.load(model_path))

    preds = predict(net, test_loader, gpu)

    results = pd.DataFrame({'Predicted': preds})
    results.index.name = 'Id'
    results.to_csv(save_path)

In [ ]:
get_results(net, test_loader, gpu, '/content/model.dat', 'bertbase_test.csv')

In [ ]:
!ls

bertbase_test.csv      drive	    test.csv	     train_text_bertplay.csv
dev_label.csv	       model.dat    test_text.csv    v2.csv
dev_text_bertplay.csv  sample_data  train_label.csv


In [ ]:
!cp -r v2.csv '/content/drive/MyDrive/Colab_Notebooks'

In [ ]:
'''
nn.Linear(768, 704),
                         nn.ReLU(),
                         nn.Dropout(p=0.1),
                         nn.Linear(704,640),
                         nn.ReLU(),
                         nn.Dropout(p=0.2),
                         nn.Linear(640,576),
                         nn.ReLU(),
                         nn.Dropout(p=0.1),
                         nn.Linear(576, 512),
                         nn.ReLU(),
                         nn.Dropout(p=0.2),
                         nn.Linear(512, 448),
                         nn.ReLU(),
                         nn.Dropout(p=0.1),
                         nn.Linear(448, 384),
                         nn.ReLU(),
                         nn.Dropout(p=0.1),
                         nn.Linear(384, 320),
                         nn.ReLU(),
                         nn.Dropout(p=0.2),
                         nn.Linear(320, 256),
                         nn.ReLU(),
                         nn.Dropout(p=0.1),
                         nn.Linear(256, 192),
                         nn.ReLU(),
                         nn.Dropout(p=0.2),
                         nn.Linear(192, 128),
                         nn.ReLU(),
                         nn.Dropout(p=0.1),
                         nn.Linear(128, 64),
                         nn.ReLU(),
                         nn.Dropout(p=0.1),
                         nn.Linear(64, 32),
                         nn.ReLU(),
                         nn.Dropout(p=0.4),
                         nn.Linear(32, 32),
                         nn.ReLU(),
                         nn.Dropout(p=0.4),
                         nn.Linear(32, 32),
                         nn.ReLU(),
                         nn.Dropout(p=0.4),                       
                         nn.Linear(32,32),
                         nn.ReLU(),
                         nn.Dropout(p=0.4),
                         nn.Linear(32,16),
                         nn.ReLU(),
                         nn.Dropout(p=0.4),
                         nn.Linear(16,16),
                         nn.ReLU(),
                         nn.Dropout(p=0.4),
                         nn.Linear(16,16),
                         nn.ReLU(),
                         nn.Dropout(p=0.4),
                         nn.Linear(16,16),
                         nn.ReLU(),
                         nn.Dropout(p=0.4),
                         nn.Linear(16,1),
'''

'\nnn.Linear(768, 704),\n                         nn.ReLU(),\n                         nn.Dropout(p=0.1),\n                         nn.Linear(704,640),\n                         nn.ReLU(),\n                         nn.Dropout(p=0.2),\n                         nn.Linear(640,576),\n                         nn.ReLU(),\n                         nn.Dropout(p=0.1),\n                         nn.Linear(576, 512),\n                         nn.ReLU(),\n                         nn.Dropout(p=0.2),\n                         nn.Linear(512, 448),\n                         nn.ReLU(),\n                         nn.Dropout(p=0.1),\n                         nn.Linear(448, 384),\n                         nn.ReLU(),\n                         nn.Dropout(p=0.1),\n                         nn.Linear(384, 320),\n                         nn.ReLU(),\n                         nn.Dropout(p=0.2),\n                         nn.Linear(320, 256),\n                         nn.ReLU(),\n                         nn.Dropout(p

In [ ]:
'''
nn.Linear(768, 256),
                         nn.ReLU(),
                         nn.Dropout(p=0.1),
                         nn.Linear(256,200),
                         nn.ReLU(),
                         nn.Dropout(p=0.1),
                         nn.Linear(200,128),
                         nn.ReLU(),
                         nn.Dropout(p=0.1),
                         nn.Linear(128, 100),
                         nn.ReLU(),
                         nn.Dropout(p=0.1),
                         nn.Linear(100, 64),
                         nn.ReLU(),
                         nn.Dropout(p=0.1),
                         nn.Linear(64, 32),
                         nn.ReLU(),
                         nn.Dropout(p=0.1),
                         nn.Linear(32, 128),
                         nn.ReLU(),
                         nn.Dropout(p=0.1),
                         nn.Linear(128, 64),
                         nn.ReLU(),
                         nn.Dropout(p=0.1),
                         nn.Linear(64, 64),
                         nn.ReLU(),
                         nn.Dropout(p=0.1),
                         nn.Linear(64, 32),
                         nn.ReLU(),
                         nn.Dropout(p=0.1),                       
                         nn.Linear(32,16),
                         nn.ReLU(),
                         nn.Dropout(p=0.1),
                         nn.Linear(16,1)
                         '''

'\nnn.Linear(768, 256),\n                         nn.ReLU(),\n                         nn.Dropout(p=0.1),\n                         nn.Linear(256,200),\n                         nn.ReLU(),\n                         nn.Dropout(p=0.1),\n                         nn.Linear(200,128),\n                         nn.ReLU(),\n                         nn.Dropout(p=0.1),\n                         nn.Linear(128, 100),\n                         nn.ReLU(),\n                         nn.Dropout(p=0.1),\n                         nn.Linear(100, 64),\n                         nn.ReLU(),\n                         nn.Dropout(p=0.1),\n                         nn.Linear(64, 32),\n                         nn.ReLU(),\n                         nn.Dropout(p=0.1),\n                         nn.Linear(32, 128),\n                         nn.ReLU(),\n                         nn.Dropout(p=0.1),\n                         nn.Linear(128, 64),\n                         nn.ReLU(),\n                         nn.Dropout(p=0.1)